# Os dados

Lendo e visualizandos os dados do Brasil do COVID até 

In [1]:

import pandas as pd

# reading csv file  
BRdata = pd.read_csv("./PG_IMT/DadosEpidemia/CoVid19.csv") 
BRdata.head()


,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,recovered,suspects,tests,tests_per_100k_inhabitants
0,2020-02-25,Brazil,SP,TOTAL,0,0,1,1,0,0,0.0,0.00218,0.0,NaN,NaN,NaN,NaN
1,2020-02-25,Brazil,TOTAL,TOTAL,0,0,1,1,0,0,0.0,0.00048,0.0,NaN,NaN,NaN,NaN
2,2020-02-26,Brazil,SP,TOTAL,0,0,0,1,0,1,0.0,0.00218,0.0,NaN,NaN,NaN,NaN
3,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,1,0,1,0.0,0.00048,0.0,NaN,NaN,NaN,NaN
4,2020-02-27,Brazil,SP,TOTAL,0,0,0,1,0,1,0.0,0.00218,0.0,NaN,NaN,NaN,NaN


## Filtrando e limpando os dados

In [2]:

is_SP =  BRdata['state']=='TOTAL'
SPdata = BRdata[is_SP]

# SPdata = SPdata.fillna(0)

SPLimpo = SPdata[SPdata.recovered.notnull()]
SPLimpo.head()


,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,recovered,suspects,tests,tests_per_100k_inhabitants
321,2020-03-23,Brazil,TOTAL,TOTAL,9,34,358,1952,34,1891,0.01618,0.92887,0.01742,8.0,15867.0,NaN,NaN
349,2020-03-24,Brazil,TOTAL,TOTAL,13,47,303,2255,46,2201,0.02237,1.07306,0.02084,20.0,17700.0,NaN,NaN
377,2020-03-25,Brazil,TOTAL,TOTAL,12,59,311,2566,57,2433,0.02808,1.22105,0.02299,27.0,27227.0,NaN,NaN
405,2020-03-26,Brazil,TOTAL,TOTAL,18,77,424,2990,77,2915,0.03664,1.42281,0.02575,42.0,48793.0,NaN,NaN
433,2020-03-27,Brazil,TOTAL,TOTAL,16,93,486,3476,92,3417,0.04425,1.65408,0.02675,42.0,50684.0,NaN,NaN


In [3]:

SPsir = SPLimpo[[ "totalCases", "deaths","recovered"]].to_numpy()
# SPsir = SPdata[[ "totalCases", "deaths","recovered"]].to_numpy()

SPsir[:,0:]
SPI = SPsir[:,0]
auxM = SPsir[:,1]
SPM = auxM
auxR = SPsir[:,2]
SPR = auxR


## Visualizando a evolução

In [4]:

from bokeh.models   import Legend, ColumnDataSource, RangeTool, LinearAxis, Range1d
from bokeh.palettes import brewer, Inferno256
from bokeh.plotting import figure, show
from bokeh.layouts  import column
from bokeh.io       import output_notebook

output_notebook()

# Criando a figura 
p = figure(
    tools="hover",
    title="Evolução do COVID",
    y_axis_type="log",
    plot_width=700, 
    plot_height=500
)

# Incluindo os dados de mortes
p.line(
    range(len(SPM)), SPM,
    legend_label="Mortes",
    line_width=4,
    line_cap="round",
    color="#de425b"
)

# Incluindo os dados de infectados
p.line(
    range(len(SPI)), SPI,
    legend_label="Infectados",
    line_width=4,
    line_cap="round",
    color="#ffd885"
)

# Incluindo os dados de recuperados
p.line(
    range(len(SPR)), SPR,
    legend_label="Removidos",
    line_width=4,
    line_cap="round",
    color="#99d594"
)


p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"
p.legend.click_policy="hide"
p.legend.location = "bottom_right"

show(p)

Loading BokehJS ...

# O problema

O conjunto de equações diferenciais que caracteriza o modelo é descrito abaixo. No modelo $\beta - \text{representa a taxa de transmissão ou taxa efetiva de contato} $  e $r - \text{a taxa de remoção ou recuperação.}$ 


$$ \begin{split}
   \frac{dS(t)}{dt} & = -\beta S(t) I(t) \\
   \frac{dI(t)}{dt} & = \beta S(t) I(t) - rI(t)  \\
   \frac{dR(t)}{dt} & = r I(t)
   \end{split}$$

   Gostaríamos de identificar quais parâmetros $\beta$ e $r$ resultam num melhor ajuste do modelo para os dados de **S**,**I** e **R**

In [5]:

# Tamanho da população - N
N = 220e6

# Importando o modelo SIR
from models import *

sir_model = ss.SIR()


Loading BokehJS ...

# Estimando os parâmetros

Para estimarmos os parâmetros do modelo $\mathbf{\beta}$ e $\mathbf{r}$, vamos utilizar  inicialmente o método de mínimos quadrados. Podemos então formular o problema a partir da Equação abaixo. Na Equação $y_m(k)$ representa o dado real em cada amostra $k$; $y_s(\theta,k)$ representa o **valor estimado** a partir da simulação do modelo para uma determinada amostra $k$ e $\theta$ representa o vetor ed parâmetros $\theta = [ \beta \; \; r]^T$. 

$$ min_{\theta}= \sum_{k=1}^{K}(y_m(k) - y_s(\theta,k))^2 $$

A equação formula a pergunta: quais os valores de $beta$ e $r$ que minizam o erro quadrático quando comparados com os dados reais.

In [6]:

import numpy as np

SPS = N - SPI - SPR

time = np.linspace(0, len(SPI), len(SPI))

# Estimando os parâmetros
sir_model.fit(SPS, SPI, SPR, time, beta_sens=[1000,1], r_sens=[100,10])


	 ├─ S(0) ─   219998040.0   I(0) ─   1952.0   R(0) ─   8.0
	 ├─ beta ─   4.545495041683099e-09   r ─   0.024390243902439025
	 ├─ beta bound ─   4.545495041683098e-12  ─  4.545495041683099e-09
	 ├─ r bound ─   0.00024390243902439024  ─  0.24390243902439024
	 ├─ equation weights ─   [0.0004435994066128953, 1, 1]
	 └─ Defined at:  5.379563434742203e-10  ─  0.019913448830013433 



In [7]:

initial = [SPS[0], SPI[0], SPR[0]]

results = sir_model.predict(initial, time)


In [8]:

# Criando a figura 
p = figure(
    tools="hover",
    title="Evolução do COVID",
    y_axis_type="log",
    plot_width=700, 
    plot_height=500
)

# Incluindo os dados de mortes
p.line(
    range(len(SPM)), SPM,
    legend_label="Mortes",
    line_width=3,
    line_cap="round",
    color="#de425b"
)

# Incluindo os dados de infectados
p.line(
    range(len(SPI)), SPI,
    legend_label="Infectados",
    line_width=3,
    line_cap="round",
    color="#ffd885"
)

# Incluindo os dados de recuperados
p.line(
    range(len(SPR)), SPR,
    legend_label="Removidos",
    line_width=3,
    line_cap="round",
    color="#99d594"
)


p.grid.grid_line_alpha = 0
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
p.yaxis.axis_label = "Indivíduos"
p.xaxis.axis_label = "Dias"
p.legend.click_policy="hide"
p.legend.location = "bottom_right"


# Incluindo os dados de infectados
p.line(
    range(len(SPI)), results[1],
    legend_label="Infectados - Modelo",
    line_width=4,
    line_dash="dashed",
    line_cap="round",
    color="#f57f17"
)

# Incluindo os dados de recuperados
p.line(
    range(len(SPR)), results[2],
    legend_label="Removidos - Modelo",
    line_dash="dashed",
    line_width=4,
    line_cap="round",
    color="#1b5e20"
)

show(p)


# Referências


- [Predictive Monitoring of COVID-19](https://ddi.sutd.edu.sg)

- [Apple mobility data](https://www.apple.com/covid19/mobility)

- [Corona Virus - Brazil Data](https://painel.covid19br.org)

- [Fitting model to Corona Virus](https://towardsdatascience.com/infectious-disease-modelling-fit-your-model-to-coronavirus-data-2568e672dbc7)